In [1]:
import pandas as pd
import numpy as np
path = '../data/'
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
viz_path = '../viz/'
from utils import *
from network import *
from geo_exploration import *

In [2]:
df = load_data(path)
df = preprocess(df)



In [3]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,hour_started,minute_started,second_started,year_ended,month_ended,day_ended,weekday_ended,hour_ended,minute_ended,second_ended
295082,11AA3AB3ED9FEFC5,classic_bike,2022-01-01 00:10:20,2022-01-01 00:19:29,Riverview Park,JC057,Riverview Park,JC057,40.744319,-74.043991,...,0,10,20,2022,1,1,5,0,19,29
312873,554FD61E00599BE2,electric_bike,2022-01-01 00:10:24,2022-01-01 00:14:06,Sip Ave,JC056,Glenwood Ave,JC094,40.730897,-74.063913,...,0,10,24,2022,1,1,5,0,14,6
312934,4D3D324F756B7103,electric_bike,2022-01-01 00:10:28,2022-01-01 00:14:12,Sip Ave,JC056,Glenwood Ave,JC094,40.730897,-74.063913,...,0,10,28,2022,1,1,5,0,14,12
295511,E26BB19DBC834216,classic_bike,2022-01-01 00:10:30,2022-01-01 00:24:30,Grove St PATH,JC005,Newport Pkwy,JC008,40.719586,-74.043117,...,0,10,30,2022,1,1,5,0,24,30
298413,DC18B042AA415411,classic_bike,2022-01-01 00:11:31,2022-01-01 00:30:43,Columbus Dr at Exchange Pl,JC106,Columbus Dr at Exchange Pl,JC106,40.716870,-74.032810,...,0,11,31,2022,1,1,5,0,30,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672735,6596578C32752C21,classic_bike,2023-12-31 23:49:41,2023-12-31 23:55:31,Lafayette Park,JC078,Grove St PATH,JC115,40.713535,-74.062828,...,23,49,41,2023,12,31,6,23,55,31
670769,51DD5B9BEFC08717,classic_bike,2023-12-31 23:49:44,2023-12-31 23:55:33,Lafayette Park,JC078,Grove St PATH,JC115,40.713584,-74.062881,...,23,49,44,2023,12,31,6,23,55,33
697644,8A6FA5AFC2CC7EC8,classic_bike,2023-12-31 23:50:04,2024-01-01 00:28:11,2 St & Park Ave,HB608,4 St & River St,HB611,40.739261,-74.032899,...,23,50,4,2024,1,1,0,0,28,11
695044,E191A2E264EB2890,classic_bike,2023-12-31 23:52:52,2023-12-31 23:55:14,Brunswick St,JC023,Newark Ave,JC032,40.724176,-74.050656,...,23,52,52,2023,12,31,6,23,55,14


In [101]:




def predictor_extraction(df):
    # Create outgoing bikes data
    outgoing = df.groupby(['start_station_name', 'year_started', 'month_started', 'weekday_started', 'hour_started']).size().reset_index(name='outgoing_bikes')

    # Create incoming bikes data
    incoming = df.groupby(['end_station_name', 'year_ended', 'month_ended', 'weekday_ended', 'hour_ended']).size().reset_index(name='incoming_bikes')
 
    outgoing.rename(columns={'start_station_name': 'station_name', 'year_started': 'year', 'month_started': 'month', 'weekday_started': 'weekday', 'hour_started': 'hour'}, inplace=True)
    incoming.rename(columns={'end_station_name': 'station_name', 'year_ended': 'year', 'month_ended': 'month', 'weekday_ended': 'weekday', 'hour_ended': 'hour'}, inplace=True)

    # Merge the dataframes
    demand = pd.merge(outgoing, incoming, how='outer', on=['station_name', 'year', 'month', 'weekday', 'hour'])
    demand.fillna(0, inplace=True)
    demand['net_bikes'] = demand['incoming_bikes'] - demand['outgoing_bikes']

    #  # Calculate cumulative bikes in use by station and timeframe
    # group_fields = ['station_name', 'year', 'month', 'weekday']
    # demand['cumulative_bikes'] = demand.groupby(group_fields)['net_bikes'].cumsum()

    # # Aggregate to find peak demands
    # peak_demand = demand.groupby(group_fields).agg({'cumulative_bikes': ['min', 'max']}).reset_index()
    # peak_demand.columns = ['station_name', 'year', 'month', 'weekday', 'Min Cumulative Bikes', 'Max Cumulative Bikes']
    
    return demand
    






def feature_extraction(df):
    # Create a feature dataframe




    network_feature_df = graph_analysis(df)

    return df

In [102]:
demand = predictor_extraction(df)

In [104]:
demand

,station_name,year,month,weekday,hour,outgoing_bikes,incoming_bikes,net_bikes
0,1 Ave & E 16 St,2023,5,2,19,0.0,1.0,1.0
1,1 Ave & E 30 St,2022,7,2,5,0.0,1.0,1.0
2,1 Ave & E 30 St,2023,9,1,8,1.0,1.0,0.0
3,1 Ave & E 6 St,2023,8,4,18,0.0,1.0,1.0
4,1 Ave & E 62 St,2022,5,2,16,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...
279862,York St & Marin Blvd,2023,12,6,18,0.0,2.0,2.0
279863,York St & Marin Blvd,2023,12,6,19,0.0,1.0,1.0
279864,York St & Marin Blvd,2023,12,6,20,2.0,1.0,-1.0
279865,York St & Marin Blvd,2023,12,6,21,3.0,1.0,-2.0


In [120]:
peak_hour = demand.groupby(['year', 'month', 'weekday', 'hour'])['net_bikes'].sum().reset_index()

In [121]:
peak_hour.sort_values('net_bikes', ascending=False).head(10)

,year,month,weekday,hour,net_bikes
2611,2023,4,3,19,124.0
836,2022,5,6,20,122.0
861,2022,6,0,21,121.0
1244,2022,8,2,20,118.0
1149,2022,7,5,21,115.0
3595,2023,10,2,19,113.0
2755,2023,5,2,19,112.0
1220,2022,8,1,20,108.0
1316,2022,8,5,20,107.0
2707,2023,5,0,19,106.0


In [92]:
demand.groupby(['year', 'month', 'weekday', 'hour']).agg({'net_bikes': 'sum'}).reset_index()

,year,month,weekday,hour,net_bikes
0,2022,1,0,0,-1.0
1,2022,1,0,1,1.0
2,2022,1,0,2,1.0
3,2022,1,0,3,2.0
4,2022,1,0,4,-1.0
...,...,...,...,...,...
4029,2023,12,6,21,1.0
4030,2023,12,6,22,6.0
4031,2023,12,6,23,-5.0
4032,2024,1,0,0,7.0


In [55]:
cul_demand['max_cumulative_bikes'].min()

-126.0

In [122]:
# Assuming df has columns for bike check-outs and check-ins with timestamps

# Calculate total check-outs per hour across all stations
hourly_demand = df.groupby(['year_started', 'month_started', 'weekday_started', 'hour_started']).size().reset_index(name='Hourly Outgoing')

# Find the maximum hourly demand which represents the peak number of bikes needed at one time
peak_hourly_demand = hourly_demand['Hourly Outgoing'].max()

In [123]:
peak_hourly_demand

2233